# Ezvan Jean-Loup #

### Mnist CNN ###

Documentation :


2) The number of kernels I chose :

For the first layer it was empirical I started off with 10 and the error rate was too high so I decided to increase that number.


The input sizes for each layer were 28, 12 and 4.
So i chose 24, 56 and 168 as number of kernels because 24*(28/12)=56 and 56*(12/4)=168.
I don't have a lot of theoretical knowledge on CNNs but I thought remembering doing it that way was legit.

3) Using poolsize=(1,1) is as if there was no pooling.

4) Changed everything from tanh to relu.


6) Transformed images from colour to gray and from 32*32 to 28*28 so it fits the CNN we created for mnist, and from the 50000 images in the training set we took 25% of them for validation set. To choose which dataset we work on you have to type the name of the dataset in evaluate_lenet5(dataset=...) But the results I had after first iterations show that there is a problem since we had an error rate as good as predicting randomly...


7) As data augmentation i added dropout, blurring and a 180° rotation.
 

In [153]:
import theano
from theano import tensor as T
from theano.tensor.nnet import conv2d
import cv2
import numpy as np
import six.moves.cPickle as pickle
import gzip
import os
import sys
import timeit
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10

In [154]:
def relu(x):
    return theano.tensor.switch(x<0, 0, x)

In [155]:


def gradient_updates_momentum(cost, params, learning_rate, momentum):
    assert momentum < 1 and momentum >= 0
    # List of update steps for each parameter
    updates = []

    for param in params:
        # For each parameter, we'll create a previous_step shared variable.
        # This variable will keep track of the parameter's update step across iterations.
        # We initialize it to 0
        previous_step = theano.shared(param.get_value()*0., broadcastable=param.broadcastable)
        # Each parameter is updated by taking a step in the direction of the gradient.
        # However, we also "mix in" the previous step according to the given momentum value.
        step = momentum*previous_step - learning_rate*T.grad(cost, param)
        # Add an update to store the previous step value
        updates.append((previous_step, step))
        # Add an update to apply the gradient descent step to the parameter itself
        updates.append((param, param + step))
    return updates



In [156]:
def load_data(dataset):
    ''' Loads the dataset

    :type dataset: string
    :param dataset: the path to the dataset (here MNIST)
    '''

    #############
    # LOAD DATA #
    #############
    if dataset=='cifar':
        
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

        
        
  # At first to make cifar data fit with my CNN I changed it to same format as MNIST but grave mistake      
        
 #       cifarnew_train= np.vstack([
 #   [
 #       cv2.resize(cv2.cvtColor(a, cv2.COLOR_BGR2GRAY),(28,28))

#
#    ]
#        for a in X_train])
        
        
        
        
#        cifarnew_test= np.vstack([
#    [
#        cv2.resize(cv2.cvtColor(a, cv2.COLOR_BGR2GRAY),(28,28))
#

 #   ]
 #       for a in X_test])
        

        X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
        
        
        
        
        train_set=(X_train.reshape(X_train.shape[0],32*32,3),y_train.reshape(y_train.shape[0]))
        valid_set=(X_validation.reshape(X_validation.shape[0],32*32,3),y_validation.reshape(y_validation.shape[0]))
        test_set=(X_test.reshape(X_test.shape[0],32*32,3),y_test.reshape(y_test.shape[0]))
        
        #print(X_train.shape,y_train.shape,X_validation.shape,y_validation.shape,cifarnew_test.shape,y_test.shape)
    
    else:
        dataset='mnist.pkl.gz'

        # Download the MNIST dataset if it is not present
        data_dir, data_file = os.path.split(dataset)
        if data_dir == "" and not os.path.isfile(dataset):
            # Check if dataset is in the data directory.
            new_path = os.path.join(
                os.path.split(__file__)[0],
                "..",
                "data",
                dataset
            )
            if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
                dataset = new_path

        if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
            from six.moves import urllib
            origin = (
                'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
            )
            print('Downloading data from %s' % origin)
            urllib.request.urlretrieve(origin, dataset)

        print('... loading data')

        # Load the dataset
        with gzip.open(dataset, 'rb') as f:
            try:
                train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
            except:
                train_set, valid_set, test_set = pickle.load(f)
    # train_set, valid_set, test_set format: tuple(input, target)
    # input is a numpy.ndarray of 2 dimensions (a matrix)
    # where each row corresponds to an example. target is a
    # numpy.ndarray of 1 dimension (vector) that has the same length as
    # the number of rows in the input. It should give the target
    # to the example with the same index in the input.
    
    
    if dataset=='cifar':
        rotato= np.vstack([
    [
        cv2.warpAffine(a.reshape(32,32,3),cv2.getRotationMatrix2D((16,16),180,1),(32,32))


    ]
        for a in train_set[0]])


        kernel = np.ones((5,5),np.float32)/25


        blur= np.vstack([
        [
            cv2.filter2D(a.reshape(32,32,3),-1,kernel)


        ]
        for a in train_set[0]])

        rotato=rotato.reshape(rotato.shape[0],32*32,3)
        blur=blur.reshape(blur.shape[0],32*32,3)
        print(train_set[0].shape,train_set[1].shape,rotato.shape,blur.shape)

        ts1=np.concatenate((train_set[0],rotato,blur),axis=0)
        ts2=np.concatenate((train_set[1],train_set[1],train_set[1]),axis=0)
        train_set0=[ts1,ts2]
        train_set=np.random.shuffle(train_set0)
        train_set=(train_set0[0],train_set0[1])
    
    else:
        dataset='mnist.pkl.gz'
        rotato= np.vstack([
    [
        cv2.warpAffine(a.reshape(28,28),cv2.getRotationMatrix2D((14,14),180,1),(28,28))


    ]
        for a in train_set[0]])


        kernel = np.ones((5,5),np.float32)/25


        blur= np.vstack([
        [
            cv2.filter2D(a.reshape(28,28),-1,kernel)


        ]
        for a in train_set[0]])

        rotato=rotato.reshape(rotato.shape[0],784)
        blur=blur.reshape(blur.shape[0],784)
        print(train_set[0].shape,train_set[1].shape,rotato.shape,blur.shape)

        ts1=np.concatenate((train_set[0],rotato,blur),axis=0)
        ts2=np.concatenate((train_set[1],train_set[1],train_set[1]),axis=0)
        ts2.reshape(ts2.shape[0],1)

        ts3=np.concatenate((ts1,ts2.reshape(ts2.shape[0],1)),axis=1)

        np.random.shuffle(ts3)
        
        ts1=ts3[:,:28*28]
        ts2=ts3[:,-1]
        train_set=(ts1,ts2)


    def shared_dataset(data_xy, borrow=True):
        """ Function that loads the dataset into shared variables

        The reason we store our dataset in shared variables is to allow
        Theano to copy it into the GPU memory (when code is run on GPU).
        Since copying data into the GPU is slow, copying a minibatch everytime
        is needed (the default behaviour if the data is not in a shared
        variable) would lead to a large decrease in performance.
        """
        data_x, data_y = data_xy
        shared_x = theano.shared(np.asarray(data_x,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        shared_y = theano.shared(np.asarray(data_y,
                                               dtype=theano.config.floatX),
                                 borrow=borrow)
        # When storing data on the GPU it has to be stored as floats
        # therefore we will store the labels as ``floatX`` as well
        # (``shared_y`` does exactly that). But during our computations
        # we need them as ints (we use labels as index, and if they are
        # floats it doesn't make sense) therefore instead of returning
        # ``shared_y`` we will have to cast it to int. This little hack
        # lets ous get around this issue
        return shared_x, T.cast(shared_y, 'int32')

    print(test_set[0].shape,test_set[1].shape,valid_set[0].shape,valid_set[1].shape,train_set[0].shape,train_set[1].shape)
    
    test_set_x, test_set_y = shared_dataset(test_set)
    valid_set_x, valid_set_y = shared_dataset(valid_set)
    train_set_x, train_set_y = shared_dataset(train_set)

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
            (test_set_x, test_set_y)]
    
    
    return rval


In [157]:
class LogisticRegression(object):
    """Multi-class Logistic Regression Class

    The logistic regression is fully described by a weight matrix :math:`W`
    and bias vector :math:`b`. Classification is done by projecting data
    points onto a set of hyperplanes, the distance to which is used to
    determine a class membership probability.
    """

    def __init__(self, input, n_in, n_out):
        """ Initialize the parameters of the logistic regression

        :type input: theano.tensor.TensorType
        :param input: symbolic variable that describes the input of the
                      architecture (one minibatch)

        :type n_in: int
        :param n_in: number of input units, the dimension of the space in
                     which the datapoints lie

        :type n_out: int
        :param n_out: number of output units, the dimension of the space in
                      which the labels lie

        """
        # start-snippet-1
        # initialize with 0 the weights W as a matrix of shape (n_in, n_out)
        self.W = theano.shared(
            value=np.zeros(
                (n_in, n_out),
                dtype=theano.config.floatX
            ),
            name='W',
            borrow=True
        )
        # initialize the biases b as a vector of n_out 0s
        self.b = theano.shared(
            value=np.zeros(
                (n_out,),
                dtype=theano.config.floatX
            ),
            name='b',
            borrow=True
        )

        # symbolic expression for computing the matrix of class-membership
        # probabilities
        # Where:
        # W is a matrix where column-k represent the separation hyperplane for
        # class-k
        # x is a matrix where row-j  represents input training sample-j
        # b is a vector where element-k represent the free parameter of
        # hyperplane-k
        self.p_y_given_x = T.nnet.softmax(T.dot(input, self.W) + self.b)

        # symbolic description of how to compute prediction as class whose
        # probability is maximal
        self.y_pred = T.argmax(self.p_y_given_x, axis=1)
        # end-snippet-1

        # parameters of the model
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

    def negative_log_likelihood(self, y):
        """Return the mean of the negative log-likelihood of the prediction
        of this model under a given target distribution.

        .. math::

            \frac{1}{|\mathcal{D}|} \mathcal{L} (\theta=\{W,b\}, \mathcal{D}) =
            \frac{1}{|\mathcal{D}|} \sum_{i=0}^{|\mathcal{D}|}
                \log(P(Y=y^{(i)}|x^{(i)}, W,b)) \\
            \ell (\theta=\{W,b\}, \mathcal{D})

        :type y: theano.tensor.TensorType
        :param y: corresponds to a vector that gives for each example the
                  correct label

        Note: we use the mean instead of the sum so that
              the learning rate is less dependent on the batch size
        """
        # start-snippet-2
        # y.shape[0] is (symbolically) the number of rows in y, i.e.,
        # number of examples (call it n) in the minibatch
        # T.arange(y.shape[0]) is a symbolic vector which will contain
        # [0,1,2,... n-1] T.log(self.p_y_given_x) is a matrix of
        # Log-Probabilities (call it LP) with one row per example and
        # one column per class LP[T.arange(y.shape[0]),y] is a vector
        # v containing [LP[0,y[0]], LP[1,y[1]], LP[2,y[2]], ...,
        # LP[n-1,y[n-1]]] and T.mean(LP[T.arange(y.shape[0]),y]) is
        # the mean (across minibatch examples) of the elements in v,
        # i.e., the mean log-likelihood across the minibatch.
        return -T.mean(T.log(self.p_y_given_x)[T.arange(y.shape[0]), y])
        # end-snippet-2

    def errors(self, y):
        """Return a float representing the number of errors in the minibatch
        over the total number of examples of the minibatch ; zero one
        loss over the size of the minibatch

        :type y: theano.tensor.TensorType
        :param y: corresponds to a vector that gives for each example the
                  correct label
        """

        # check if y has same dimension of y_pred
        if y.ndim != self.y_pred.ndim:
            raise TypeError(
                'y should have the same shape as self.y_pred',
                ('y', y.type, 'y_pred', self.y_pred.type)
            )
        # check if y is of the correct datatype
        if y.dtype.startswith('int'):
            # the T.neq operator returns a vector of 0s and 1s, where 1
            # represents a mistake in prediction
            return T.mean(T.neq(self.y_pred, y))
        else:
            raise NotImplementedError()



In [158]:
class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, W=None, b=None,
                 activation=relu):
        """
        Typical hidden layer of a MLP: units are fully-connected and have
        sigmoidal activation function. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        NOTE : The nonlinearity used here is tanh

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.dmatrix
        :param input: a symbolic tensor of shape (n_examples, n_in)

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: theano.Op or function
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        self.input = input
        # end-snippet-1

        # `W` is initialized with `W_values` which is uniformely sampled
        # from sqrt(-6./(n_in+n_hidden)) and sqrt(6./(n_in+n_hidden))
        # for tanh activation function
        # the output of uniform if converted using asarray to dtype
        # theano.config.floatX so that the code is runable on GPU
        # Note : optimal initialization of weights is dependent on the
        #        activation function used (among other things).
        #        For example, results presented in [Xavier10] suggest that you
        #        should use 4 times larger initial weights for sigmoid
        #        compared to tanh
        #        We have no info for other function, so we use the same as
        #        tanh.
        if W is None:
            W_values = np.asarray(
                rng.uniform(
                    low=-np.sqrt(6. / (n_in + n_out)),
                    high=np.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                dtype=theano.config.floatX
            )
            if activation == theano.tensor.nnet.sigmoid:
                W_values *= 4

            W = theano.shared(value=W_values, name='W', borrow=True)

        if b is None:
            b_values = np.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b', borrow=True)

        self.W = W
        self.b = b

        lin_output = T.dot(input, self.W) + self.b
        self.output = (
            lin_output if activation is None
            else activation(lin_output)
        )
        # parameters of the model
        self.params = [self.W, self.b]



In [159]:
from __future__ import print_function

import os
import sys
import timeit


import theano
import theano.tensor as T
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d





class LeNetConvPoolLayer(object):
    """Pool Layer of a convolutional network """

    def __init__(self, rng, input, filter_shape, image_shape, poolsize=(2, 2)):
        """
        Allocate a LeNetConvPoolLayer with shared variable internal parameters.

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type input: theano.tensor.dtensor4
        :param input: symbolic image tensor, of shape image_shape

        :type filter_shape: tuple or list of length 4
        :param filter_shape: (number of filters, num input feature maps,
                              filter height, filter width)

        :type image_shape: tuple or list of length 4
        :param image_shape: (batch size, num input feature maps,
                             image height, image width)

        :type poolsize: tuple or list of length 2
        :param poolsize: the downsampling (pooling) factor (#rows, #cols)
        """

        assert image_shape[1] == filter_shape[1]
        self.input = input

        # there are "num input feature maps * filter height * filter width"
        # inputs to each hidden unit
        fan_in = np.prod(filter_shape[1:])
        # each unit in the lower layer receives a gradient from:
        # "num output feature maps * filter height * filter width" /
        #   pooling size
        fan_out = (filter_shape[0] * np.prod(filter_shape[2:]) //
                   np.prod(poolsize))
        # initialize weights with random weights
        W_bound = np.sqrt(6. / (fan_in + fan_out))
        self.W = theano.shared(
            np.asarray(
                rng.uniform(low=-W_bound, high=W_bound, size=filter_shape),
                dtype=theano.config.floatX
            ),
            borrow=True
        )

        # the bias is a 1D tensor -- one bias per output feature map
        b_values = np.zeros((filter_shape[0],), dtype=theano.config.floatX)
        self.b = theano.shared(value=b_values, borrow=True)

        # convolve input feature maps with filters
        conv_out = conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            input_shape=image_shape
        )

        # pool each feature map individually, using maxpooling
        pooled_out = pool.pool_2d(
            input=conv_out,
            ds=poolsize,
            ignore_border=True
        )

        # add the bias term. Since the bias is a vector (1D array), we first
        # reshape it to a tensor of shape (1, n_filters, 1, 1). Each bias will
        # thus be broadcasted across mini-batches and feature map
        # width & height
        self.output = relu(pooled_out + self.b.dimshuffle('x', 0, 'x', 'x'))

        # store parameters of this layer
        self.params = [self.W, self.b]

        # keep track of model input
        self.input = input

##Choose dataset='cifar' or dataset='mnist.pkl.gz'

def evaluate_lenet5(learning_rate=0.1, n_epochs=20,
                    dataset='mnist.pkl.gz',
                    nkerns=[24, 56,168], batch_size=500):
    """ Demonstrates lenet on MNIST dataset

    :type learning_rate: float
    :param learning_rate: learning rate used (factor for the stochastic
                          gradient)

    :type n_epochs: int
    :param n_epochs: maximal number of epochs to run the optimizer

    :type dataset: string
    :param dataset: path to the dataset used for training /testing (MNIST here)

    :type nkerns: list of ints
    :param nkerns: number of kernels on each layer
    """
    
    rng = np.random.RandomState(23455)
    srng = theano.tensor.shared_randomstreams.RandomStreams(rng.randint(999999))
    dropout=0.5
    
    #The momentum has to be between 0 and 1
    momentum=0.9
        
    datasets = load_data(dataset)

    train_set_x, train_set_y = datasets[0]
    valid_set_x, valid_set_y = datasets[1]
    test_set_x, test_set_y = datasets[2]

    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.shape[0]
    n_valid_batches = valid_set_x.shape[0]
    n_test_batches = test_set_x.shape[0]
    n_train_batches //= batch_size
    n_valid_batches //= batch_size
    n_test_batches //= batch_size

    # allocate symbolic variables for the data
    index = T.lscalar()  # index to a [mini]batch
    #momentum= T.lscalar()
    #learning_rate=T.lscalar()

    #create momentum and learning rate as T.lscalar()
    
    # start-snippet-1
  
    x = T.matrix('x')   # the data is presented as rasterized images
        
    y = T.ivector('y')  # the labels are presented as 1D vector of
                        # [int] labels
    #y=T.matrix('y')
    
    if dataset=='cifar':
        print('cifar')
        
    else:
        print('mnist')

    ######################
    # BUILD ACTUAL MODEL #
    ######################
    print('... building the model')
    

    
        
  

    # Reshape matrix of rasterized images of shape (batch_size, 28 * 28)
    # to a 4D tensor, compatible with our LeNetConvPoolLayer
    # (28, 28) is the size of MNIST images.
    
    if dataset=='cifar':
        layer0_input = x.reshape((batch_size, 3, 32, 32))

    # Construct the first convolutional pooling layer:
    # filtering reduces the image size to (32-5+1 , 32-5+1) = (28, 28)
    # maxpooling reduces this further to (28/2, 28/2) = (14, 14)
    # 4D output tensor is thus of shape (batch_size, nkerns[0], 14, 14)
        layer0 = LeNetConvPoolLayer(
            rng,
            input=layer0_input,
            image_shape=(batch_size, 3, 32, 32),
            filter_shape=(nkerns[0], 3, 5, 5),
            poolsize=(2, 2)
        )


    # Construct the second convolutional pooling layer
    # filtering reduces the image size to (14-5+1, 14-5+1) = (10, 10)
    # maxpooling reduces this further to (10/2, 10/2) = (5, 5)
    # 4D output tensor is thus of shape (batch_size, nkerns[1], 5, 5)
        layer1 = LeNetConvPoolLayer(
            rng,
            input=layer0.output,
            image_shape=(batch_size, nkerns[0], 14, 14),
            filter_shape=(nkerns[1], nkerns[0], 5, 5),
            poolsize=(2, 2)
        )



    # We add a third convolutionnal layer without pooling and filter size = 3*3
    # The filtering reduces the image size to (5-3+1, 5-3+1)=(3, 3)
    # 4D output tensor is thus of shape(batch_size, nkerns[2], 3, 3)

        layer2 = LeNetConvPoolLayer(
            rng,
            input=layer1.output,
            image_shape=(batch_size, nkerns[1], 5, 5),
            filter_shape=(nkerns[2], nkerns[1], 3, 3),
            poolsize=(1, 1)
        )

    # the HiddenLayer being fully-connected, it operates on 2D matrices of
    # shape (batch_size, num_pixels) (i.e matrix of rasterized images).
    # This will generate a matrix of shape (batch_size, nkerns[2] * 3 * 3),
    # or (500, 168 * 3 * 3) = (500, 1512) with the default values.
        layer3_input = layer2.output.flatten(2)

        if dropout > 0 :
            mask = srng.binomial(n=1, p=1-dropout, size=layer3_input.shape)
            layer3_input = layer3_input * T.cast(mask, theano.config.floatX)



    # construct a fully-connected sigmoidal layer
        layer3 = HiddenLayer(
            rng,
            input=layer3_input,
            n_in=nkerns[2] * 3 * 3,
            n_out=500,
            activation=relu
        )

        if dropout > 0 :
            mask = srng.binomial(n=1, p=1-dropout, size=layer3.output.shape)
            layer3.output = layer3.output * T.cast(mask, theano.config.floatX)

    # classify the values of the fully-connected sigmoidal layer
        layer4 = LogisticRegression(input=layer3.output, n_in=500, n_out=10)
        
        
    else:
        layer0_input = x.reshape((batch_size, 1, 28, 28))

    # Construct the first convolutional pooling layer:
    # filtering reduces the image size to (28-5+1 , 28-5+1) = (24, 24)
    # maxpooling reduces this further to (24/2, 24/2) = (12, 12)
    # 4D output tensor is thus of shape (batch_size, nkerns[0], 12, 12)
        layer0 = LeNetConvPoolLayer(
            rng,
            input=layer0_input,
            image_shape=(batch_size, 1, 28, 28),
            filter_shape=(nkerns[0], 1, 5, 5),
            poolsize=(2, 2)
        )


    # Construct the second convolutional pooling layer
    # filtering reduces the image size to (12-5+1, 12-5+1) = (8, 8)
    # maxpooling reduces this further to (8/2, 8/2) = (4, 4)
    # 4D output tensor is thus of shape (batch_size, nkerns[1], 4, 4)
        layer1 = LeNetConvPoolLayer(
            rng,
            input=layer0.output,
            image_shape=(batch_size, nkerns[0], 12, 12),
            filter_shape=(nkerns[1], nkerns[0], 5, 5),
            poolsize=(2, 2)
        )



    # We add a third convolutionnal layer without pooling and filter size = 3*3
    # The filtering reduces the image size to (4-3+1, 4-3+1)=(2, 2)
    # 4D output tensor is thus of shape(batch_size, nkerns[2], 2, 2)

        layer2 = LeNetConvPoolLayer(
            rng,
            input=layer1.output,
            image_shape=(batch_size, nkerns[1], 4, 4),
            filter_shape=(nkerns[2], nkerns[1], 3, 3),
            poolsize=(1, 1)
        )

    # the HiddenLayer being fully-connected, it operates on 2D matrices of
    # shape (batch_size, num_pixels) (i.e matrix of rasterized images).
    # This will generate a matrix of shape (batch_size, nkerns[2] * 2 * 2),
    # or (500, 168 * 2 * 2) = (500, 672) with the default values.
        layer3_input = layer2.output.flatten(2)

        if dropout > 0 :
            mask = srng.binomial(n=1, p=1-dropout, size=layer3_input.shape)
            layer3_input = layer3_input * T.cast(mask, theano.config.floatX)



    # construct a fully-connected sigmoidal layer
        layer3 = HiddenLayer(
            rng,
            input=layer3_input,
            n_in=nkerns[2] * 2 * 2,
            n_out=500,
            activation=relu
        )

        if dropout > 0 :
            mask = srng.binomial(n=1, p=1-dropout, size=layer3.output.shape)
            layer3.output = layer3.output * T.cast(mask, theano.config.floatX)

    # classify the values of the fully-connected sigmoidal layer
        layer4 = LogisticRegression(input=layer3.output, n_in=500, n_out=10)

    # the cost we minimize during training is the NLL of the model
    cost = layer4.negative_log_likelihood(y)

    # create a function to compute the mistakes that are made by the model


    test_model = theano.function(
        [index],
        layer4.errors(y),
        givens={
            x: test_set_x[index * batch_size: (index + 1) * batch_size],
            y: test_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    validate_model = theano.function(
        [index],
        layer4.errors(y),
        givens={
            x: valid_set_x[index * batch_size: (index + 1) * batch_size],
            y: valid_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )

    # create a list of all model parameters to be fit by gradient descent
    params = layer4.params + layer3.params + layer2.params + layer1.params + layer0.params

    # create a list of gradients for all model parameters
    grads = T.grad(cost, params)

    # train_model is a function that updates the model parameters by
    # SGD Since this model has many parameters, it would be tedious to
    # manually create an update rule for each model parameter. We thus
    # create the updates list by automatically looping over all
    # (params[i], grads[i]) pairs.
    #updates = [
    #    (param_i, param_i - learning_rate * grad_i)
    #    for param_i, grad_i in zip(params, grads)
    #]
    #New updates with a momentum to the gradient descent
    updates=gradient_updates_momentum(cost, params, learning_rate, momentum)
    
    
    train_model = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size],
            y: train_set_y[index * batch_size: (index + 1) * batch_size]
        }
    )
    # end-snippet-1

    ###############
    # TRAIN MODEL #
    ###############
    print('... training')
    # early-stopping parameters
    patience = 10000  # look as this many examples regardless
    patience_increase = 2  # wait this much longer when a new best is
                           # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = min(n_train_batches.eval(), patience // 2)
                                  # go through this many
                                  # minibatche before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = np.inf
    best_iter = 0
    test_score = 0.
    start_time = timeit.default_timer()

    epoch = 0
    done_looping = False

    while (epoch < n_epochs) and (not done_looping):
        epoch = epoch + 1
        #if epoch%10==0:
            #learning_rate=learning_rate*0.95
            #With that formula we know momentum is gonna increase closely towards 1 without ever reaching it
            #momentum=momentum*(1+5*(1-momentum)/100)
            
            #set_value for learning rate and momentum
        for minibatch_index in range(n_train_batches.eval()):

            iter = (epoch - 1) * n_train_batches.eval() + minibatch_index

            if iter % 100 == 0:
                print('training @ iter = ', iter)
            cost_ij = train_model(minibatch_index)

            if (iter + 1) % validation_frequency == 0:

                # compute zero-one loss on validation set
                validation_losses = [validate_model(i) for i
                                     in range(n_valid_batches.eval())]
                this_validation_loss = np.mean(validation_losses)
                print('epoch %i, minibatch %i/%i, validation error %f %%' %
                      (epoch, minibatch_index + 1, n_train_batches.eval(),
                       this_validation_loss * 100.))

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:

                    #improve patience if loss improvement is good enough
                    if this_validation_loss < best_validation_loss *  \
                       improvement_threshold:
                        patience = max(patience, iter * patience_increase)

                    # save best validation score and iteration number
                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    # test it on the test set
                    test_losses = [
                        test_model(i)
                        for i in range(n_test_batches.eval())
                    ]
                    test_score = np.mean(test_losses)
                    print(('     epoch %i, minibatch %i/%i, test error of '
                           'best model %f %%') %
                          (epoch, minibatch_index + 1, n_train_batches.eval(),
                           test_score * 100.))

            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()
    print('Optimization complete.')
    print('Best validation score of %f %% obtained at iteration %i, '
          'with test performance %f %%' %
          (best_validation_loss * 100., best_iter + 1, test_score * 100.))

    print(' ran for %.2fm' % ((end_time - start_time) / 60.))

if __name__ == '__main__':
    evaluate_lenet5()


def experiment(state, channel):
    evaluate_lenet5(state.learning_rate, dataset=state.dataset)

... loading data
(50000, 784) (50000,) (50000, 784) (50000, 784)
(10000, 784) (10000,) (10000, 784) (10000,) (150000, 784) (150000,)
mnist
... building the model


C:\Users\jean-\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.


... training
training @ iter =  0
training @ iter =  100
training @ iter =  200
epoch 1, minibatch 300/300, validation error 8.020000 %
     epoch 1, minibatch 300/300, test error of best model 7.750000 %
training @ iter =  300
training @ iter =  400
training @ iter =  500
epoch 2, minibatch 300/300, validation error 5.510000 %
     epoch 2, minibatch 300/300, test error of best model 5.150000 %
training @ iter =  600
training @ iter =  700
training @ iter =  800
epoch 3, minibatch 300/300, validation error 5.140000 %
     epoch 3, minibatch 300/300, test error of best model 4.650000 %
training @ iter =  900
training @ iter =  1000
training @ iter =  1100
epoch 4, minibatch 300/300, validation error 4.020000 %
     epoch 4, minibatch 300/300, test error of best model 3.620000 %
training @ iter =  1200
training @ iter =  1300
training @ iter =  1400
epoch 5, minibatch 300/300, validation error 3.610000 %
     epoch 5, minibatch 300/300, test error of best model 3.670000 %
training @ ite

In [160]:
z=[(10000, 784), (10000,) ,(10000, 784) ,(10000,) ,(150000, 784), (150000,)]